In [ ]:
#| default_exp handlers.helcom

# HELCOM

> Data pipeline (handler) to convert HELCOM data ([source](https://helcom.fi/about-us)) to `NetCDF` format or `Open Refine` format.  

<!-- ## HELCOM MORS Environment database -->

[Helcom MORS data](https://helcom.fi/about-us) is provided as a Microsoft Access database. 
[`Mdbtools`](https://github.com/mdbtools/mdbtools) can be used to convert the tables of the Microsoft Access database to `.csv` files on Unix-like OS.

Example steps:
1. Download data (e.g. https://metadata.helcom.fi/geonetwork/srv/fin/catalog.search#/metadata/2fdd2d46-0329-40e3-bf96-cb08c7206a24). 
2. Install mdbtools via VScode Terminal 

    ```
    sudo apt-get -y install mdbtools
    ````

3. Install unzip via VScode Terminal 

    ```
    sudo apt-get -y install unzip
    ````

4. In VS code terminal, navigate to the marisco data folder

    ```
    cd /home/marisco/downloads/marisco/_data/accdb/mors_19840101_20211231
    ```

5. Unzip MORS_ENVIRONMENT.zip 

    ```
    unzip MORS_ENVIRONMENT.zip 
    ```

6. Run preprocess.sh to generate the required data files

    ```
    ./preprocess.sh MORS_ENVIRONMENT.zip
    ````
7. Conetens of 'preprocess.sh' script.
    ```
    #!/bin/bash

    # Example of use: ./preprocess.sh MORS_ENVIRONMENT.zip
    unzip $1
    dbname=$(ls *.accdb)
    mkdir csv
    for table in $(mdb-tables -1 "$dbname"); do
        echo "Export table $table"
        mdb-export "$dbname" "$table" > "csv/$table.csv"
    done
    ```

***

## Packages import

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import pandas as pd # Python package that provides fast, flexible, and expressive data structures.
import numpy as np
from tqdm import tqdm # Python Progress Bar Library
from functools import partial # Function which Return a new partial object which when called will behave like func called with the positional arguments args and keyword arguments keywords
import fastcore.all as fc # package that brings fastcore functionality, see https://fastcore.fast.ai/.
from pathlib import Path # This module offers classes representing filesystem paths
from dataclasses import asdict

from marisco.utils import (has_valid_varname, match_worms, match_maris_lut, Match)
from marisco.callbacks import (Callback, Transformer, EncodeTimeCB, SanitizeLonLatCB)
from marisco.metadata import (GlobAttrsFeeder, BboxCB, DepthRangeCB, TimeRangeCB, ZoteroCB, KeyValuePairCB)
from marisco.configs import (base_path, nc_tpl_path, cfg, cache_path, cdl_cfg, Enums, lut_path,
                             species_lut_path, sediments_lut_path, bodyparts_lut_path, 
                             detection_limit_lut_path, filtered_lut_path, area_lut_path)
from marisco.serializers import NetCDFEncoder
from collections.abc import Callable
from math import modf
import warnings
from marisco.netcdf_to_csv import (LookupTimeFromEncodedTime, GetSampleTypeCB,
                                   LookupNuclideByIdCB, ConvertLonLatCB, LookupUnitByIdCB,
                                   LookupValueTypeByIdCB, LookupSpeciesByIdCB, 
                                   LookupBodypartByIdCB, LookupSedimentTypeByIdCB)                                  
from marisco.serializers import OpenRefineCsvEncoder

In [ ]:
warnings.filterwarnings('ignore')

***

##  MARIS NetCDF 
When MARISCO is installed, it uses `cdl.toml` to create the `maris-template.nc`, which acts as a standardized template for MARIS NetCDF files. The `cdl.toml` is a configuration file listing all the variables allowed in the NetCDF4 files. The contents of the cdl.toml can be retrieved with the function `cdl_cfg()`.  

Retrieving the keys of the `cdl_cfg()`.

In [ ]:
print (cdl_cfg()['vars'].keys())

dict_keys(['defaults', 'bio', 'sed', 'suffixes'])


Printing the contents of all keys

In [ ]:
print (cdl_cfg()['vars']['defaults'].keys())
print (cdl_cfg()['vars']['bio'].keys())
print (cdl_cfg()['vars']['sed'].keys())
print (cdl_cfg()['vars']['suffixes'].keys())

dict_keys(['lon', 'lat', 'smp_depth', 'tot_depth', 'time', 'area'])
dict_keys(['bio_group', 'species', 'body_part'])
dict_keys(['sed_type'])
dict_keys(['uncertainty', 'detection_limit', 'volume', 'salinity', 'temperature', 'filtered', 'counting_method', 'sampling_method', 'preparation_method', 'unit'])


***

## MARIS Open Refine 

Currently, updates to the MARIS database are facilitated through a standardized CSV file using Open Refine. Description of the variables included in this CSV file are provided at [Maris](https://maris.iaea.org/help/1132).
 



***

## MARIS Open Refine CSV & MARIS NetCDF variable relationship. 

The table below lists the MARIS variables in both MARIS Open Refine and MARIS NetCDF formats. Each variable's presence in both formats for the seawater (``sea``), biota (``bio``), and sediment (``sed``) groups is indicated with a checkmark (``✓``).


<style>
  table {
    width: 100%;
    border-collapse: collapse
  }

  td,
  th {
    border: 1px solid #000;
    padding: 5px;
    text-align: center
  }

  th {
    background-color: #f2f2f2
  }

  .open-refine {
    background-color: #fff;
    color: black;
    text-align: center

  }

  .netcdf {
    background-color: #e6e6e6;
    color: black;
    text-align: center
  }
</style>
<table>
  <thead>
    <tr>
      <th class="open-refine">Open Refine Variables</th>
      <th class="open-refine">sea</th>
      <th class="open-refine">bio</th>
      <th class="open-refine">sed</th>
      <th class="netcdf">sea</th>
      <th class="netcdf">bio</th>
      <th class="netcdf">sed</th>
      <th class="netcdf">NetCDF Variables</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td class="open-refine">Sample type</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">*Included as netcdf.group*</td>
    </tr>
    <tr>
      <td class="open-refine">Latitude decimal</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">lat</td>
    </tr>
    <tr>
      <td class="open-refine">Longitude decimal</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">lon</td>
    </tr>
    <tr>
      <td class="open-refine">Sampling start date</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">time</td>
    </tr>
    <tr>
      <td class="open-refine">Sampling start time</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">time</td>
    </tr>
    <tr>
      <td class="open-refine">Sampling end date</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Sampling end time</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Nuclide</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">nuclide</td>
    </tr>
    <tr>
      <td class="open-refine">Value type</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">detection_limit</td>
    </tr>
    <tr>
      <td class="open-refine">Unit</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">unit</td>
    </tr>
    <tr>
      <td class="open-refine">Activity or MDA</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">value</td>
    </tr>
    <tr>
      <td class="open-refine">Uncertainty</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">uncertainty</td>
    </tr>
    <tr>
      <td class="open-refine">Sampling depth</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">smp_depth</td>
    </tr>
    <tr>
      <td class="open-refine">Top</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Bottom</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Species</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="netcdf"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf">species</td>
    </tr>
    <tr>
      <td class="open-refine">Body part</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="netcdf"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf">body_part</td>
    </tr>
    <tr>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf">bio_group</td>
    </tr>
    <tr>
      <td class="open-refine">Salinity</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">salinity</td>
    </tr>
    <tr>
      <td class="open-refine">Temperature</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">temperature</td>
    </tr>
    <tr>
      <td class="open-refine">Filtered</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">filtered</td>
    </tr>
    <tr>
      <td class="open-refine">Mesh size</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Quality flag</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Sediment type</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">sed_type</td>
    </tr>
    <tr>
      <td class="open-refine">Dry weight</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Wet weight</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Dry/wet ratio</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Station ID</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Sample ID</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">data_provider_sample_id</td>
    </tr>
    <tr>
      <td class="open-refine">Total depth</td>
      <td class="open-refine">✓</td>
      <td class="open-refine"></td>
      <td class="open-refine"></td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">tot_depth</td>
    </tr>
    <tr>
      <td class="open-refine">Profile or transect ID</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Sampling method</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">sampling_method</td>
    </tr>
    <tr>
      <td class="open-refine">Preparation method</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">preparation_method</td>
    </tr>
    <tr>
      <td class="open-refine">Drying method</td>
      <td class="open-refine"></td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
      <td class="netcdf"></td>
    </tr>
    <tr>
      <td class="open-refine">Counting method</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">counting_method</td>
    </tr>
    <tr>
      <td class="open-refine">Sample notes</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">sample_notes<sup>*1</sup></td>
    </tr>
    <tr>
      <td class="open-refine">Measurement notes</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="open-refine">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">✓</td>
      <td class="netcdf">measurement_notes<sup>*1</sup></td>
    </tr>
  </tbody>
</table>

<sup>*1</sup> The MARIS NetCDF does not currently support strings of variable length (i.e. vlen string data type).

***

## Define variables

1. **fname_in** - is the path to the folder containing the HELCOM data in CSV format. The path can be defined as a relative path. 

2. **fname_out_nc** - is the path and filename for the NetCDF output.The path can be defined as a relative path. 

3. **Zotero key** - is used to retrieve attributes related to the dataset from [Zotero](https://www.zotero.org/). The MARIS datasets include a [library](https://maris.iaea.org/datasets) available on [Zotero](https://www.zotero.org/groups/2432820/maris/library). 


In [ ]:
# | export
fname_in = '../../_data/accdb/mors/csv'
fname_out_nc = '../../_data/output/100-HELCOM-MORS-2024.nc'
fname_out_csv = '../../_data/output/100-HELCOM-MORS-2024.csv'
zotero_key ='26VMZZ2Q'
ref_id = 100

***

## Utils

In [ ]:
#| export
def load_data(src_dir,
                smp_types=['SEA', 'SED', 'BIO']):
    "Load HELCOM data and return the data in a dictionary of dataframes with the dictionary key as the sample type"
    dfs = {}
    lut_smp_type = {'SEA': 'seawater', 'SED': 'sediment', 'BIO': 'biota'}
    for smp_type in smp_types:
        fname_meas = smp_type + '02.csv' # measurement (i.e. radioactivity) information.
        fname_smp = smp_type + '01.csv' # sample information 
        df = pd.merge(pd.read_csv(Path(src_dir)/fname_meas),  # measurements
                      pd.read_csv(Path(src_dir)/fname_smp),  # sample
                      on='KEY', how='left')
        dfs[lut_smp_type[smp_type]] = df
    return dfs

***

## Load data

`dfs` is a dictionary of dataframes created from the Helcom dataset located at the path `fname_in`. The data to be included in each dataframe is sorted by sample type. Each dictionary is defined with a key equal to the sample type. 

In [ ]:
#|eval: false
dfs = load_data(fname_in)
print(dfs.keys())
print(f"Seawater cols: {dfs['seawater'].columns}")
print(f"Sediment cols: {dfs['sediment'].columns}")
print(f"Biota cols: {dfs['biota'].columns}")

dict_keys(['seawater', 'sediment', 'biota'])
Seawater cols: Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/m³', 'VALUE_Bq/m³', 'ERROR%_m³',
       'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR',
       'MONTH', 'DAY', 'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'TDEPTH', 'SDEPTH', 'SALIN',
       'TTEMP', 'FILT', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'DATE_OF_ENTRY_y'],
      dtype='object')
Sediment cols: Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'ERROR%_kg',
       '< VALUE_Bq/m²', 'VALUE_Bq/m²', 'ERROR%_m²', 'DATE_OF_ENTRY_x',
       'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY',
       'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'DEVICE', 'TDEPTH',
       'UPPSLI', 'LOWSLI', 'AREA', 'SEDI', 'OXIC', 'DW%', 'LOI%',
       'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'SUM_LINK', 'DATE_OF_ENTRY_y'],
      dty

Show the structure of the `seawater` dataframe:

In [ ]:
#|eval: false
dfs['seawater'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/m³,VALUE_Bq/m³,ERROR%_m³,DATE_OF_ENTRY_x,COUNTRY,LABORATORY,SEQUENCE,...,LONGITUDE (ddmmmm),LONGITUDE (dddddd),TDEPTH,SDEPTH,SALIN,TTEMP,FILT,MORS_SUBBASIN,HELCOM_SUBBASIN,DATE_OF_ENTRY_y
0,WKRIL2012003,CS137,NaN,NaN,5.3,32.0,08/20/14 00:00:00,90.0,KRIL,2012003.0,...,29.20,29.3333,NaN,0.0,NaN,NaN,NaN,11.0,11.0,08/20/14 00:00:00
1,WKRIL2012004,CS137,NaN,NaN,19.9,20.0,08/20/14 00:00:00,90.0,KRIL,2012004.0,...,29.20,29.3333,NaN,29.0,NaN,NaN,NaN,11.0,11.0,08/20/14 00:00:00
2,WKRIL2012005,CS137,NaN,NaN,25.5,20.0,08/20/14 00:00:00,90.0,KRIL,2012005.0,...,23.09,23.1500,NaN,0.0,NaN,NaN,NaN,11.0,3.0,08/20/14 00:00:00
3,WKRIL2012006,CS137,NaN,NaN,17.0,29.0,08/20/14 00:00:00,90.0,KRIL,2012006.0,...,27.59,27.9833,NaN,0.0,NaN,NaN,NaN,11.0,11.0,08/20/14 00:00:00
4,WKRIL2012007,CS137,NaN,NaN,22.2,18.0,08/20/14 00:00:00,90.0,KRIL,2012007.0,...,27.59,27.9833,NaN,39.0,NaN,NaN,NaN,11.0,11.0,08/20/14 00:00:00


Show the structure of the `biota` dataframe:

In [ ]:
#|eval: false
dfs['biota'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,BASIS,ERROR%,NUMBER,DATE_OF_ENTRY_x,COUNTRY,...,BIOTATYPE,TISSUE,NO,LENGTH,WEIGHT,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,DATE_OF_ENTRY_y
0,BVTIG2012041,CS134,VTIG01,<,0.010140,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2.0,16,02/27/14 00:00:00
1,BVTIG2012041,K40,VTIG01,,135.300000,W,3.57,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2.0,16,02/27/14 00:00:00
2,BVTIG2012041,CO60,VTIG01,<,0.013980,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2.0,16,02/27/14 00:00:00
3,BVTIG2012041,CS137,VTIG01,,4.338000,W,3.48,NaN,02/27/14 00:00:00,6.0,...,F,5,16.0,45.7,948.0,18.453,92.9,2.0,16,02/27/14 00:00:00
4,BVTIG2012040,CS134,VTIG01,<,0.009614,W,NaN,NaN,02/27/14 00:00:00,6.0,...,F,5,17.0,45.9,964.0,18.458,92.9,2.0,16,02/27/14 00:00:00


Show the structure of the `sediment` dataframe: 

In [ ]:
#|eval: false
dfs['sediment'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,ERROR%_kg,< VALUE_Bq/m²,VALUE_Bq/m²,ERROR%_m²,DATE_OF_ENTRY_x,...,LOWSLI,AREA,SEDI,OXIC,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,SUM_LINK,DATE_OF_ENTRY_y
0,SKRIL2012048,RA226,NaN,NaN,35.0,26.0,NaN,NaN,NaN,08/20/14 00:00:00,...,20.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
1,SKRIL2012049,RA226,NaN,NaN,36.0,22.0,NaN,NaN,NaN,08/20/14 00:00:00,...,27.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
2,SKRIL2012050,RA226,NaN,NaN,38.0,24.0,NaN,NaN,NaN,08/20/14 00:00:00,...,2.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
3,SKRIL2012051,RA226,NaN,NaN,36.0,25.0,NaN,NaN,NaN,08/20/14 00:00:00,...,4.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00
4,SKRIL2012052,RA226,NaN,NaN,30.0,23.0,NaN,NaN,NaN,08/20/14 00:00:00,...,6.0,0.006,NaN,NaN,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00


***

## Data transformation pipeline for NetCDF.

### Data transformation pipeline utils

``CompareDfsAndTfm`` compares the original dataframes to the transformed dataframe. A dictionary of dataframes, ``tfm.dfs_dropped``, is created to include the data present in the original dataset but absent from the transformed data. ``tfm.compare_stats`` provides a quick overview of the number of rows in both the original dataframes and the transformed dataframe.

In [ ]:
# | export
class CompareDfsAndTfm(Callback):
    "Create a dfs of dropped data. Data included in the DFS not in the TFM"
    def __init__(self, dfs_compare):
        fc.store_attr()

    def __call__(self, tfm):
        tfm.dfs_dropped={}
        tfm.compare_stats={}
        for grp in tfm.dfs.keys():
           
            # get the index values in dfs (i.e. dfs_compare) not in tfm.dfs. 
            index_diff=self.dfs_compare[grp].index.difference(tfm.dfs[grp].index)
            tfm.dfs_dropped[grp] = self.dfs_compare[grp].loc[index_diff]

            tfm.compare_stats[grp]= {'Number of rows in dfs :' : len(self.dfs_compare[grp].index),
                                     'Number of rows in tfm.dfs:' : len(tfm.dfs[grp].index),
                                     'Number of dropped rows:' : len(tfm.dfs_dropped[grp].index),
                                     'Number of rows in tfm.dfs + Number of dropped rows:' : len(tfm.dfs[grp].index) + len(tfm.dfs_dropped[grp].index)
                                    }

### Normalize nuclide names

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``nuclide``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Nuclide``.*

#### Lower & strip nuclide names

Create a callback function, `LowerStripRdnNameCB`, that receives a dictionary of dataframes. For each dataframe in the dictionary, it converts the contents of the `Nuclides` column to lowercase and removes any leading or trailing whitespace.

In [ ]:
#| export
class LowerStripRdnNameCB(Callback):
    "Convert nuclide names to lowercase & strip any trailing space(s)"
    def __call__(self, tfm):
        for k in tfm.dfs.keys():
            tfm.dfs[k]['NUCLIDE'] = tfm.dfs[k]['NUCLIDE'].apply(
                lambda x: x.lower().strip())

Here we call a transformer, which applies the callback (e.g. `LowerStripRdnNameCB`) to the dictionary of dataframes, `dfs`. We then print the unique entries of the transformed `NUCLIDE` column for each dataframe included in the dictionary of dataframes, `dfs`.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB()])
print('seawater nuclides: ')
print(tfm()['seawater']['NUCLIDE'].unique())
print('biota nuclides: ')
print(tfm()['biota']['NUCLIDE'].unique())
print('sediment nuclides: ')
print(tfm()['sediment']['NUCLIDE'].unique())

seawater nuclides: 
['cs137' 'sr90' 'h3' 'cs134' 'pu238' 'pu239240' 'am241' 'cm242' 'cm244'
 'tc99' 'k40' 'ru103' 'sr89' 'sb125' 'nb95' 'ru106' 'zr95' 'ag110m'
 'cm243244' 'ba140' 'ce144' 'u234' 'u238' 'co60' 'pu239' 'pb210' 'po210'
 'np237' 'pu240' 'mn54']
biota nuclides: 
['cs134' 'k40' 'co60' 'cs137' 'sr90' 'ag108m' 'mn54' 'co58' 'ag110m'
 'zn65' 'sb125' 'pu239240' 'ru106' 'be7' 'ce144' 'pb210' 'po210' 'sb124'
 'sr89' 'zr95' 'te129m' 'ru103' 'nb95' 'ce141' 'la140' 'i131' 'ba140'
 'pu238' 'u235' 'bi214' 'pb214' 'pb212' 'tl208' 'ac228' 'ra223' 'eu155'
 'ra226' 'gd153' 'sn113' 'fe59' 'tc99' 'co57' 'sn117m' 'eu152' 'sc46'
 'rb86' 'ra224' 'th232' 'cs134137' 'am241' 'ra228' 'th228' 'k-40' 'cs138'
 'cs139' 'cs140' 'cs141' 'cs142' 'cs143' 'cs144' 'cs145' 'cs146']
sediment nuclides: 
['ra226' 'cs137' 'ra228' 'k40' 'sr90' 'cs134137' 'cs134' 'pu239240'
 'pu238' 'co60' 'ru103' 'ru106' 'sb125' 'ag110m' 'ce144' 'am241' 'be7'
 'th228' 'pb210' 'co58' 'mn54' 'zr95' 'ba140' 'po210' 'ra224' 'nb95'
 'p

#### Remap HELCOM nuclide names to MARIS nuclide names

The `maris-template.nc` file, which  is created from the `cdl.toml` on installation of the Marisco package, provides details of the nuclides permitted in the  MARIS NetCDF file. Here we define a function  `get_unique_nuclides()` which creates a list of the unique nuclides from each dataframe in the dictionary of dataframes `dfs`. The function `has_valid_varname` checks that each nuclide in this list is included in the `maris-template.nc` (i.e. the `cdl.toml`). `has_valid_varname` returns all variables in the list that are not in the `maris-template.nc` or returns `True`. 
 

In [ ]:
#| export
def get_unique_nuclides(dfs):
    "Get list of unique radionuclide types measured across samples."
    nuclides = []
    for k in dfs.keys():
        nuclides += dfs[k]['NUCLIDE'].unique().tolist()
    # remove duplicates from nuclides list.
    nuclides=list(set(nuclides))
    return nuclides

In [ ]:
#|eval: false
# Check if these variable names are consistent with MARIS CDL
has_valid_varname(get_unique_nuclides(tfm.dfs), nc_tpl_path())

"pu238240" variable name not found in MARIS CDL
"cs142" variable name not found in MARIS CDL
"cs145" variable name not found in MARIS CDL
"cs139" variable name not found in MARIS CDL
"cs134137" variable name not found in MARIS CDL
"cs140" variable name not found in MARIS CDL
"cs144" variable name not found in MARIS CDL
"cs138" variable name not found in MARIS CDL
"pu239240" variable name not found in MARIS CDL
"cs141" variable name not found in MARIS CDL
"cm243244" variable name not found in MARIS CDL
"cs143" variable name not found in MARIS CDL
"cs146" variable name not found in MARIS CDL
"k-40" variable name not found in MARIS CDL


False

Many nuclide names are not listed in the `maris-template.nc`. Here we create a look up table, `varnames_lut_updates`, which will be used to correct the nuclide names in the dictionary of dataframes (i.e. dfs) that are not compatible with the `maris-template.nc`.

In [ ]:
#| export
varnames_lut_updates = {
    'k-40': 'k40',
    'cm243244': 'cm243_244_tot',
    'cs134137': 'cs134_137_tot',
    'pu239240': 'pu239_240_tot',
    'pu238240': 'pu238_240_tot',
    'cs138': 'cs137',
    'cs139': 'cs137',
    'cs140': 'cs137',
    'cs141': 'cs137',
    'cs142': 'cs137',
    'cs143': 'cs137',
    'cs144': 'cs137',
    'cs145': 'cs137',
    'cs146': 'cs137'}

Function `get_varnames_lut` returns a dictionary of nuclide names. This dictionary includes the `NUCLIDE` names from the dataframes in dfs, along with corrections specified in `varnames_lut_updates`.

In [ ]:
#| export
def get_varnames_lut(dfs, lut=varnames_lut_updates):
    lut = {n: n for n in set(get_unique_nuclides(dfs))}
    lut.update(varnames_lut_updates)
    return lut

Create a callback that remaps the nuclide names in the dataframes within dfs to the updated names in `varnames_lut_updates`.

In [ ]:
# | export
class RemapRdnNameCB(Callback):
    "Remap to MARIS radionuclide names."
    def __init__(self,
                 fn_lut=partial(get_varnames_lut, lut=varnames_lut_updates)):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut(tfm.dfs)
        for grp in tfm.dfs.keys():
            tfm.dfs[grp]['NUCLIDE'].replace(lut, inplace=True)

Apply the transformer for callbacks `LowerStripRdnNameCB` and `RemapRdnNameCB`. Then, print the unique nuclides for each dataframe in the dictionary dfs.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            #CompareDfsAndTfm(dfs)
                            ])
tfm()

#print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print('seawater nuclides: ')
print(tfm.dfs['seawater']['NUCLIDE'].unique())
print('biota nuclides: ')
print(tfm.dfs['biota']['NUCLIDE'].unique())
print('sediment nuclides: ')
print(tfm.dfs['sediment']['NUCLIDE'].unique())

seawater nuclides: 
['cs137' 'sr90' 'h3' 'cs134' 'pu238' 'pu239_240_tot' 'am241' 'cm242'
 'cm244' 'tc99' 'k40' 'ru103' 'sr89' 'sb125' 'nb95' 'ru106' 'zr95'
 'ag110m' 'cm243_244_tot' 'ba140' 'ce144' 'u234' 'u238' 'co60' 'pu239'
 'pb210' 'po210' 'np237' 'pu240' 'mn54']
biota nuclides: 
['cs134' 'k40' 'co60' 'cs137' 'sr90' 'ag108m' 'mn54' 'co58' 'ag110m'
 'zn65' 'sb125' 'pu239_240_tot' 'ru106' 'be7' 'ce144' 'pb210' 'po210'
 'sb124' 'sr89' 'zr95' 'te129m' 'ru103' 'nb95' 'ce141' 'la140' 'i131'
 'ba140' 'pu238' 'u235' 'bi214' 'pb214' 'pb212' 'tl208' 'ac228' 'ra223'
 'eu155' 'ra226' 'gd153' 'sn113' 'fe59' 'tc99' 'co57' 'sn117m' 'eu152'
 'sc46' 'rb86' 'ra224' 'th232' 'cs134_137_tot' 'am241' 'ra228' 'th228']
sediment nuclides: 
['ra226' 'cs137' 'ra228' 'k40' 'sr90' 'cs134_137_tot' 'cs134'
 'pu239_240_tot' 'pu238' 'co60' 'ru103' 'ru106' 'sb125' 'ag110m' 'ce144'
 'am241' 'be7' 'th228' 'pb210' 'co58' 'mn54' 'zr95' 'ba140' 'po210'
 'ra224' 'nb95' 'pu238_240_tot' 'pu241' 'pu239' 'eu155' 'ir192' 'th2

After apply correction to the nuclide names check that all nuclide in the dictionary of dataframees are valid. Returns `True` if all are valid.

In [ ]:
#|eval: false
has_valid_varname(get_unique_nuclides(tfm.dfs), nc_tpl_path())

True

***

### Parse time

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: `time`.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: `Sampling start date` and `Sampling start time`.*

Create a callback that remaps the time format in the dictionary of dataframes (i.e. `%m/%d/%y %H:%M:%S`):

In [ ]:
#| export
class ParseTimeCB(Callback):
    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            # get 'time' from 'DATE' column
            tfm.dfs[grp]['time'] = pd.to_datetime(tfm.dfs[grp]['DATE'], format='%m/%d/%y %H:%M:%S')
            # if 'DATE' column is nan, get 'time' from 'YEAR','MONTH' and 'DAY' column. 
            # if 'DAY' or 'MONTH' is 0 then set it to 1. 
            tfm.dfs[grp].loc[tfm.dfs[grp]["DAY"] == 0, "DAY"] = 1
            tfm.dfs[grp].loc[tfm.dfs[grp]["MONTH"] == 0, "MONTH"] = 1
            
            # if 'DAY' and 'MONTH' is nan but YEAR is not nan then set 'DAY' and 'MONTH' both to 1. 
            condition = (tfm.dfs[grp]["DAY"].isna()) & (tfm.dfs[grp]["MONTH"].isna()) & (tfm.dfs[grp]["YEAR"].notna())
            tfm.dfs[grp].loc[condition, "DAY"] = 1
            tfm.dfs[grp].loc[condition, "MONTH"] = 1
            
            
            condition = tfm.dfs[grp]['DATE'].isna() # if 'DATE' is nan. 
            tfm.dfs[grp]['time']  = np.where(condition,
                                             # 'coerce', then invalid parsing will be set as NaT. NaT will result if the number of days are not valid for the month.
                                            pd.to_datetime(tfm.dfs[grp][['YEAR', 'MONTH', 'DAY']], format='%y%m%d', errors='coerce'),  
                                            pd.to_datetime(tfm.dfs[grp]['DATE'], format='%m/%d/%y %H:%M:%S'))


Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB` and `ParseTimeCB`. Then, print the `time` data for `seawater`.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            #CompareDfsAndTfm(dfs)
                            ])
tfm()
#print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['seawater']['time'])

0       2012-05-23
1       2012-05-23
2       2012-06-17
3       2012-05-24
4       2012-05-24
           ...    
21211   2021-10-15
21212   2021-11-04
21213   2021-10-15
21214   2021-05-17
21215   2021-05-13
Name: time, Length: 21216, dtype: datetime64[ns]


***

### Encode time (seconds since ...)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``time``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: `Sampling start date` and `Sampling start time`*

`EncodeTimeCB` converts the HELCOM `time` format to the MARIS `time` format.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg(), verbose = True),
                            CompareDfsAndTfm(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
                            

8 of 21216 entries for `time` are invalid for seawater.
1 of 39817 entries for `time` are invalid for sediment.
                                                    seawater  sediment  biota
Number of rows in dfs :                                21216     39817  15827
Number of rows in tfm.dfs:                             21208     39816  15827
Number of dropped rows:                                    8         1      0
Number of rows in tfm.dfs + Number of dropped r...     21216     39817  15827 



In [ ]:
tfm.dfs_dropped['sediment'][['YEAR', 'MONTH', 'DAY', 'DATE']]

,YEAR,MONTH,DAY,DATE
35821,NaN,NaN,NaN,NaN


***

### Normalize uncertainty

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``uncertainty``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: `Uncertainty`.*

Function `unc_rel2stan` coverts uncertainty from relative uncertainty to standard uncertainty.

In [ ]:
#| export
# Make measurement and uncertainty units consistent
def unc_rel2stan(df, meas_col, unc_col):
    return df.apply(lambda row: row[unc_col] * row[meas_col]/100, axis=1)

For each sample type in the Helcom dataset, the uncertainty is given as a relative uncertainty to the value (i.e., activity). The column names for both the value and the uncertainty vary by sample type. The coi_units_unc dictionary defines the column names for the Value and Uncertainty for each sample type.

In [ ]:
#| export
# Columns of interest
coi_units_unc = [('seawater', 'VALUE_Bq/m³', 'ERROR%_m³'),
                 ('biota', 'VALUE_Bq/kg', 'ERROR%'),
                 ('sediment', 'VALUE_Bq/kg', 'ERROR%_kg')]

NormalizeUncUnitCB callback normalizes the uncertainty by converting from relative uncertainty to standard uncertainty. 

In [ ]:
#| export
class NormalizeUncUnitCB(Callback):
    "Convert from relative error % to uncertainty of activity unit"
    def __init__(self, 
                 fn_convert_unc=unc_rel2stan,
                 coi=coi_units_unc):
        fc.store_attr()

    def __call__(self, tfm):
        for grp, val, unc in self.coi:
            tfm.dfs[grp][unc] = self.fn_convert_unc(tfm.dfs[grp], val, unc)

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB` and `NormalizeUncUnitCB()`. Then, print the value (i.e. activity per unit ) and standard uncertainty for each sample type.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB()])

print(tfm()['seawater'][['VALUE_Bq/m³', 'ERROR%_m³']][:5])
print(tfm()['biota'][['VALUE_Bq/kg', 'ERROR%']][:5])
print(tfm()['sediment'][['VALUE_Bq/kg', 'ERROR%_kg']][:5])

   VALUE_Bq/m³  ERROR%_m³
0          5.3      1.696
1         19.9      3.980
2         25.5      5.100
3         17.0      4.930
4         22.2      3.996
   VALUE_Bq/kg    ERROR%
0     0.010140       NaN
1   135.300000  6.535274
2     0.013980       NaN
3     4.338000  0.006549
4     0.009614       NaN
   VALUE_Bq/kg  ERROR%_kg
0         35.0   1.114750
1         36.0   1.026432
2         38.0   1.316928
3         36.0   1.166400
4         30.0   0.621000


***

### Lookup transformations 

#### Lookup MARIS function 

`get_maris_lut` performs a lookup of data provided in `data_provider_lut` against the MARIS lookup (`maris_lut`) using a fuzzy matching algorithm based on Levenshtein distance. The `get_maris_lut` is used to correct the HELCOM data to a standard format for MARIS. 

In [ ]:
#|export
def get_maris_lut(fname_in, 
                  fname_cache, # For instance 'species_helcom.pkl'
                  data_provider_lut:str, # Data provider lookup table name
                  data_provider_id_col:str, # Data provider lookup column id of interest
                  data_provider_name_col:str, # Data provider lookup column name of interest
                  maris_lut:Callable, # Function retrieving MARIS source lookup table
                  maris_id: str, # Id of MARIS lookup table nomenclature item to match
                  maris_name: str, # Name of MARIS lookup table nomenclature item to match
                  unmatched_fixes={},
                  as_dataframe=False,
                  overwrite=False
                 ):
    fname_cache = cache_path() / fname_cache
    lut = {}
    maris_lut = maris_lut()
    df = pd.read_csv(Path(fname_in) / data_provider_lut)
    if overwrite or (not fname_cache.exists()):
        for _, row in tqdm(df.iterrows(), total=len(df)):

            # Fix if unmatched
            has_to_be_fixed = row[data_provider_id_col] in unmatched_fixes            
            name_to_match = unmatched_fixes[row[data_provider_id_col]] if has_to_be_fixed else row[data_provider_name_col]

            # Match
            result = match_maris_lut(maris_lut, name_to_match, maris_id, maris_name)
            match = Match(result.iloc[0][maris_id], result.iloc[0][maris_name], 
                          row[data_provider_name_col], result.iloc[0]['score'])
            
            lut[row[data_provider_id_col]] = match
        fc.save_pickle(fname_cache, lut)
    else:
        lut = fc.load_pickle(fname_cache)

    if as_dataframe:
        df_lut = pd.DataFrame({k: asdict(v) for k, v in lut.items()}).transpose()
        df_lut.index.name = 'source_id'
        return df_lut.sort_values(by='match_score', ascending=False)
    else:
        return lut

#### Lookup : Biota species

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``species``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: `Species`.*

The HELCOM dataset includes look-up in the `RUBIN_NAME.csv` file for biota species. 

In [ ]:
#|eval: false
df_rubin = pd.read_csv(Path(fname_in) / 'RUBIN_NAME.csv')
df_rubin.head(5)

,RUBIN_ID,RUBIN,SCIENTIFIC NAME,ENGLISH NAME
0,11,ABRA BRA,ABRAMIS BRAMA,BREAM
1,12,ANGU ANG,ANGUILLA ANGUILLA,EEL
2,13,ARCT ISL,ARCTICA ISLANDICA,ISLAND CYPRINE
3,14,ASTE RUB,ASTERIAS RUBENS,COMMON STARFISH
4,15,CARD EDU,CARDIUM EDULE,COCKLE


Create `unmatched_fixes_biota_species` to correct the spelling of names that are unmatched in the HELCOM dataset. 

In [ ]:
#|export
unmatched_fixes_biota_species = {
    'CARD EDU': 'Cerastoderma edule',
    'LAMI SAC': 'Saccharina latissima',
    'PSET MAX': 'Scophthalmus maximus',
    'STIZ LUC': 'Sander luciopercas'}

In [ ]:
#|eval: false
species_lut_df = get_maris_lut(fname_in, 
                               fname_cache='species_helcom.pkl', 
                               data_provider_lut='RUBIN_NAME.csv',
                               data_provider_id_col='RUBIN',
                               data_provider_name_col='SCIENTIFIC NAME',
                               maris_lut=species_lut_path,
                               maris_id='species_id',
                               maris_name='species',
                               unmatched_fixes=unmatched_fixes_biota_species,
                               as_dataframe=True,
                               overwrite=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [00:06<00:00,  6.88it/s]


Display `species_lut_df`. The `match_score` represents the number insertions, deletions, or substitutions needed to transform from the HECOM source name (`source_name`) to the maris name, (`matched_maris_name`). 

In [ ]:
#|eval: false
species_lut_df.head()

,matched_id,matched_maris_name,source_name,match_score
source_id,,,,
ENCH CIM,276,Echinodermata,ENCHINODERMATA CIM,5
MACO BAL,122,Macoma balthica,MACOMA BALTICA,1
STUC PEC,704,Stuckenia pectinata,STUCKENIA PECTINATE,1
STIZ LUC,285,Sander lucioperca,STIZOSTEDION LUCIOPERCA,1
ABRA BRA,271,Abramis brama,ABRAMIS BRAMA,0


Show `species_lut_df` where `match_type` is not a perfect match ( i.e. not equal 0).

In [ ]:
species_lut_df[species_lut_df['match_score'] >= 1]

,matched_id,matched_maris_name,source_name,match_score
source_id,,,,
ENCH CIM,276,Echinodermata,ENCHINODERMATA CIM,5
MACO BAL,122,Macoma balthica,MACOMA BALTICA,1
STUC PEC,704,Stuckenia pectinata,STUCKENIA PECTINATE,1
STIZ LUC,285,Sander lucioperca,STIZOSTEDION LUCIOPERCA,1


`LookupBiotaSpeciesCB` applies the corrected `biota` `species` data obtained from the `get_maris_lut` function to the `biota` dataframe in the dictionary of dataframes, `dfs`.

In [ ]:
#| export
class LookupBiotaSpeciesCB(Callback):
    """
    Biota species remapped to MARIS db:
        CARD EDU: Cerastoderma edule
        LAMI SAC: Saccharina latissima
        PSET MAX: Scophthalmus maximus
        STIZ LUC: Sander luciopercas
    """
    def __init__(self, fn_lut): fc.store_attr()
    def __call__(self, tfm):
        lut = self.fn_lut()
        tfm.dfs['biota']['species'] = tfm.dfs['biota']['RUBIN'].apply(lambda x: lut[x.strip()].matched_id)

`get_maris_species` defines a partial function of `get_maris_lut`, with predefined arguments  for species lookup.

In [ ]:
#| export
get_maris_species = partial(get_maris_lut,
                            fname_in, fname_cache='species_helcom.pkl', 
                            data_provider_lut='RUBIN_NAME.csv',
                            data_provider_id_col='RUBIN',
                            data_provider_name_col='SCIENTIFIC NAME',
                            maris_lut=species_lut_path,
                            maris_id='species_id',
                            maris_name='species',
                            unmatched_fixes=unmatched_fixes_biota_species,
                            as_dataframe=False,
                            overwrite=False)

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()` and `LookupBiotaSpeciesCB(get_maris_species)`. Then, print the unique `species` for the `biota` dataframe.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species)
                            ])

#print(tfm()['biota'][['RUBIN', 'species']][:10])
print(tfm()['biota']['species'].unique())

[  99  243   50  139  270  192  191  284   84  269  122   96  287  279
  278  288  286  244  129  275  271  285  283  247  120   59  280  274
  273  290  289  272  277  276   21  282  110  281  245  704 1524  703
 1611  621   60]


***

#### Lookup : Biota tissues

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``body_part``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: `Body part`.*

The HELCOM dataset includes look-up in the `TISSUE.csv` file for biota tissues. Biota tissue is known as `body part` in the maris data set.    

In [ ]:
#|eval: false
pd.read_csv('../../_data/accdb/mors/csv/TISSUE.csv').head()

,TISSUE,TISSUE_DESCRIPTION
0,1,WHOLE FISH
1,2,WHOLE FISH WITHOUT ENTRAILS
2,3,WHOLE FISH WITHOUT HEAD AND ENTRAILS
3,4,FLESH WITH BONES
4,5,FLESH WITHOUT BONES (FILETS)


Create `unmatched_fixes_biota_tissues` to correct entries in the HELCOM dataset. 

In [ ]:
#|export
unmatched_fixes_biota_tissues = {
    3: 'Whole animal eviscerated without head',
    12: 'Viscera',
    8: 'Skin'}

In [ ]:
#|eval: false
tissues_lut_df = get_maris_lut(fname_in, 
                               fname_cache='tissues_helcom.pkl', 
                               data_provider_lut='TISSUE.csv',
                               data_provider_id_col='TISSUE',
                               data_provider_name_col='TISSUE_DESCRIPTION',
                               maris_lut=bodyparts_lut_path,
                               maris_id='bodypar_id',
                               maris_name='bodypar',
                               unmatched_fixes=unmatched_fixes_biota_tissues,
                               as_dataframe=True,
                               overwrite=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 150.02it/s]


In [ ]:
tissues_lut_df.head()

,matched_id,matched_maris_name,source_name,match_score
source_id,,,,
2,52,Flesh without bones,WHOLE FISH WITHOUT ENTRAILS,13
5,52,Flesh without bones,FLESH WITHOUT BONES (FILETS),9
1,1,Whole animal,WHOLE FISH,5
15,53,Stomach and intestine,STOMACH + INTESTINE,3
41,1,Whole animal,WHOLE ANIMALS,1


`LookupBiotaBodyPartCB` applies the corrected `biota` `TISSUE` data obtained from the `get_maris_lut` function to the `biota` dataframe in the dictionary of dataframes, `dfs`.

In [ ]:
#| export
class LookupBiotaBodyPartCB(Callback):
    """
    Update bodypart id based on MARIS dbo_bodypar.xlsx:
        - 3: 'Whole animal eviscerated without head',
        - 12: 'Viscera',
        - 8: 'Skin'
    """
    def __init__(self, fn_lut): fc.store_attr()
    def __call__(self, tfm):
        lut = self.fn_lut()
        tfm.dfs['biota']['body_part'] = tfm.dfs['biota']['TISSUE'].apply(lambda x: lut[x].matched_id)

`get_maris_bodypart` defines a partial function of `get_maris_lut`, with predefined arguments  for  `TISSUE` (or `bodypar`) lookup.

In [ ]:
#| export
get_maris_bodypart = partial(get_maris_lut,
                             fname_in,
                             fname_cache='tissues_helcom.pkl', 
                             data_provider_lut='TISSUE.csv',
                             data_provider_id_col='TISSUE',
                             data_provider_name_col='TISSUE_DESCRIPTION',
                             maris_lut=bodyparts_lut_path,
                             maris_id='bodypar_id',
                             maris_name='bodypar',
                             unmatched_fixes=unmatched_fixes_biota_tissues)

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)` and `LookupbioooooooootaBodyPartCB(get_maris_bodypart)`. Then, print the `TISSUE` and `body_part` for the `biota` dataframe.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart)
                            ])

print(tfm()['biota'][['TISSUE', 'body_part']][:5])

   TISSUE  body_part
0       5         52
1       5         52
2       5         52
3       5         52
4       5         52


***

#### Lookup : Biogroup

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``bio_group``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: Biogroup is not included.*

`get_biogroup_lut` reads the file at `species_lut_path()` and from the contents of this file creates a dictionary linking `species_id` to `biogroup_id`.

In [ ]:
#| export
def get_biogroup_lut(maris_lut):
    species = pd.read_excel(maris_lut)
    return species[['species_id', 'biogroup_id']].set_index('species_id').to_dict()['biogroup_id']

`LookupBiogroupCB` applies the corrected `biota` `bio group` data obtained from the `get_maris_lut` function to the `biota` dataframe in the dictionary of dataframes, `dfs`.

In [ ]:

#| export
class LookupBiogroupCB(Callback):
    """
    Update biogroup id  based on MARIS dbo_species.xlsx
    """
    def __init__(self, fn_lut): fc.store_attr()
    def __call__(self, tfm):
        lut = self.fn_lut()
        tfm.dfs['biota']['bio_group'] = tfm.dfs['biota']['species'].apply(lambda x: lut[x])

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)`, `LookupSedimentCB(get_maris_sediments)` and `LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())` . Then, print the `bio_group` for the `biota` dataframe.

In [ ]:

#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart),                            
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path()))
                            ])

print(tfm()['biota']['bio_group'].unique())

[ 4  2 14 11  8  3]


***

#### Lookup : Sediment types

The HELCOM dataset includes look-up in the `SEDIMENT_TYPE.csv` file for Sediment types. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``sed_type``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: `Sediment type`.*

In [ ]:
#|eval: false
df_sediment = pd.read_csv(Path(fname_in) / 'SEDIMENT_TYPE.csv')
df_sediment.head()

,SEDI,SEDIMENT TYPE,RECOMMENDED TO BE USED
0,-99,NO DATA,NaN
1,0,GRAVEL,YES
2,1,SAND,YES
3,2,FINE SAND,NO
4,3,SILT,YES


Create `unmatched_fixes_sediments` to correct entries in the HELCOM dataset. 

In [ ]:
#|export
unmatched_fixes_sediments = {
    #np.nan: 'Not applicable',
    -99: '(Not available)'
}

In [ ]:
#|eval: false
sediments_lut_df = get_maris_lut(
    fname_in, 
    fname_cache='sediments_helcom.pkl', 
    data_provider_lut='SEDIMENT_TYPE.csv',
    data_provider_id_col='SEDI',
    data_provider_name_col='SEDIMENT TYPE',
    maris_lut=sediments_lut_path,
    maris_id='sedtype_id',
    maris_name='sedtype',
    unmatched_fixes=unmatched_fixes_sediments,
    as_dataframe=True,
    overwrite=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:00<00:00, 154.13it/s]


`get_maris_sediments` defines a partial function of `get_maris_lut`, with predefined arguments  for  `SEDI` (or `sedtype`) lookup.

In [ ]:
#| export
get_maris_sediments = partial(
    get_maris_lut,
    fname_in, 
    fname_cache='sediments_helcom.pkl', 
    data_provider_lut='SEDIMENT_TYPE.csv',
    data_provider_id_col='SEDI',
    data_provider_name_col='SEDIMENT TYPE',
    maris_lut=sediments_lut_path,
    maris_id='sedtype_id',
    maris_name='sedtype',
    unmatched_fixes=unmatched_fixes_sediments)

`LookupSedimentCB` applies the corrected `sediment` `SEDI` data obtained from the `get_maris_lut` function to the `sediment` dataframe in the dictionary of dataframes, `dfs`.

In [ ]:
#| export
class LookupSedimentCB(Callback):
    """
    Update sediment id  based on MARIS dbo_sedtype.xlsx
        -99: '(Not available)'
        - na: '(Not available)'
        - 56: '(Not available)'
        - 73: '(Not available)'
    """
    def __init__(self, fn_lut): fc.store_attr()
    def __call__(self, tfm):
        lut = self.fn_lut()

        # To check with Helcom
        tfm.dfs['sediment']['SEDI'] = dfs['sediment']['SEDI'].fillna(-99).astype('int')
        tfm.dfs['sediment']['SEDI'].replace(56, -99, inplace=True)
        tfm.dfs['sediment']['SEDI'].replace(73, -99, inplace=True)
        
        tfm.dfs['sediment']['sed_type'] = tfm.dfs['sediment']['SEDI'].apply(lambda x: lut[x].matched_id)

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)` and `LookupSedimentCB(get_maris_sediments)`. Then, print the `SEDI` and `sed_type` for the `biota` dataframe.

In [ ]:

#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments)
                            ])

print(tfm()['sediment'][['SEDI', 'sed_type']][:5])
print(tfm())

   SEDI  sed_type
0   -99         0
1   -99         0
2   -99         0
3   -99         0
4   -99         0
{'seawater':                 KEY NUCLIDE METHOD < VALUE_Bq/m³  VALUE_Bq/m³     ERROR%_m³  \
0      WKRIL2012003   cs137    NaN           NaN          5.3      0.089888   
1      WKRIL2012004   cs137    NaN           NaN         19.9      0.792020   
2      WKRIL2012005   cs137    NaN           NaN         25.5      1.300500   
3      WKRIL2012006   cs137    NaN           NaN         17.0      0.838100   
4      WKRIL2012007   cs137    NaN           NaN         22.2      0.887112   
...             ...     ...    ...           ...          ...           ...   
21211  WSSSM2021005      h3  SSM45           NaN       1030.0   9888.000000   
21212  WSSSM2021006      h3  SSM45           NaN       2240.0  21728.000000   
21213  WSSSM2021007      h3  SSM45           NaN       2060.0  19982.000000   
21214  WSSSM2021008      h3  SSM45           NaN       2300.0  23000.000000   
21215  WSS

***

#### Lookup : Units

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``unit``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Unit``.*

Create `renaming_unit_rules` to rename the units. 

In [ ]:
#| export
# Define unit names renaming rules
renaming_unit_rules = {'seawater' : 1, #  'Bq/m3'
                       'sediment' : 4, # 'Bq/kgd' for sediment (see https://maps.helcom.fi/website/download/MORS_ENVIRONMENT_Reporting_form.xlsx)
                       'biota': {'D' : 4, # 'Bq/kgd'
                                 'W' : 5, # 'Bq/kgw'
                                 'F' : 5 # 'Bq/kgw' !assumed to be 'Fresh' so set to wet. .  
                                 } } 


`LookupUnitCB` defines a `unit` column each dataframe based on the units provided in the value (`VALUE_Bq/m³` or `VALUE_Bq/kg`) column of the HELCOM dataset. 

In [ ]:
#| export
class LookupUnitCB(Callback):
    def __init__(self,
                 renaming_unit_rules=renaming_unit_rules):
        fc.store_attr()
    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            
            if grp == 'biota':
                lut=renaming_unit_rules[grp]
                # lookup value in the 'BASIS' column to determine the unit. 
                tfm.dfs[grp]['unit'] = tfm.dfs[grp]['BASIS'].apply(lambda x: lut[x] if x in lut.keys() else 0 )
            else:                 
                tfm.dfs[grp]['unit'] = renaming_unit_rules[grp]

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)`, `LookupSedimentCB(get_maris_sediments)`, `LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())` and `LookupUnitCB()`. Then, print the unique `unit` for the `seawater` dataframe.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB()])

print(tfm()['biota']['unit'].unique())

[5 0 4]


***

#### Lookup : Detection limit or Value type

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``detection_limit``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine foramt variable: ``Value type``.*

Create `coi_dl` to define the column names related to Value type for each dataset. 

In [ ]:
#| export
# Columns of interest
coi_dl = {'seawater' : { 'val' : 'VALUE_Bq/m³',
                        'unc' : 'ERROR%_m³',
                        'dl' : '< VALUE_Bq/m³'},
                 'biota':  {'val' : 'VALUE_Bq/kg',
                            'unc' : 'ERROR%',
                            'dl' : '< VALUE_Bq/kg'},
                 'sediment': { 'val' : 'VALUE_Bq/kg',
                              'unc' : 'ERROR%_kg',
                              'dl' : '< VALUE_Bq/kg'}}

`get_detectionlimit_lut` reads the file at `detection_limit_lut_path()` and from the contents of this file creates a dictionary linking `name` to `id`.
| id | name | name_sanitized |
| :-: | :-: | :-: |
|-1|Not applicable|Not applicable|
|0|Not Available|Not available|
|1|=|Detected value|
|2|<|Detection limit|
|3|ND|Not detected|
|4|DE|Derived|

In [ ]:
#| export 
def get_detectionlimit_lut():
    df = pd.read_excel(detection_limit_lut_path(), usecols=['name','id'])
    return df.set_index('name').to_dict()['id']

`LookupDetectionLimitCB` creates a `detection_limit` column with values determined as follows:
1. Perform a lookup with the appropriate columns value type (or detection limit) columns (`< VALUE_Bq/m³` or `< VALUE_Bq/kg`) against the table returned from the function `get_detectionlimit_lut`.
2. If `< VALUE_Bq/m³` or `< VALUE_Bq/kg>` is NaN but both activity values (`VALUE_Bq/m³` or `VALUE_Bq/kg`) and standard uncertainty (`ERROR%_m³`, `ERROR%`, or `ERROR%_kg`) are provided, then assign the ID of `1` (i.e. "Detected value").
3. For other NaN values in the `detection_limit` column, set them to `0` (i.e. `Not Available`).

In [ ]:
# | export
class LookupDetectionLimitCB(Callback):
    "Remap value type to MARIS format."
    def __init__(self ,
                 coi=coi_dl,
                 fn_lut=get_detectionlimit_lut
                 ):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        for grp in tfm.dfs.keys():
            # Copy dl col 
            tfm.dfs[grp]['detection_limit'] = tfm.dfs[grp][self.coi[grp]['dl']]
            # Fill values with '=' if both a value and uncertainty are not nan and detection_limit is not in the list of keys returned from lut.
            condition = ((tfm.dfs[grp][self.coi[grp]['val']].notna()) & (tfm.dfs[grp][self.coi[grp]['unc']].notna())) & (~tfm.dfs[grp]["detection_limit"].isin(list(lut.keys())))
            tfm.dfs[grp].loc[condition, 'detection_limit']= '='
            # Fill values that are not in the lut with 'Not Available'.
            tfm.dfs[grp].loc[~tfm.dfs[grp]["detection_limit"].isin(list(lut.keys())), "detection_limit"] = 'Not Available'
            # Perform lookup
            tfm.dfs[grp]['detection_limit'] = tfm.dfs[grp]['detection_limit'].apply(lambda x: lut[x])
            

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)`, `LookupSedimentCB(get_maris_sediments)`, `LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())`, `LookupUnitCB()` and `LookupDetectionLimitCB`. Then, print the unique `detection_limit` for the `seawater` dataframe.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB()])

print(tfm()['seawater']['detection_limit'].unique())

[1 2 0]


***

#### Lookup : Method

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *NetCDF4 format variables: ``counting_method``, ``sampling_method`` and ``preparation_method``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Sampling method``,	``Preparation method`` and ``Counting method``.*

> 'Method' is provided in the HELCOM data but some work is required to link it to MARIS 'counting_method', 'sampling_method' and 'preparation_method'.

***

### Data provider sample id

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``data_provider_sample_id``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Sample ID``*

>  MARIS NetCDF4 format for variable type ``data_provider_sample_id`` does not support vlen strings.

In [ ]:
# | export
class RemapDataProviderSampleIdCB(Callback):
    "Remap key to MARIS data_provider_sample_id format."
    def __init__(self):
        fc.store_attr()

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            # data_provider_sample_id
            tfm.dfs[grp]['data_provider_sample_id'] = tfm.dfs[grp]['KEY']


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            CompareDfsAndTfm(dfs)
                            ])

print(tfm()['seawater']['data_provider_sample_id'].unique())
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


['WKRIL2012003' 'WKRIL2012004' 'WKRIL2012005' ... 'WSSSM2021006'
 'WSSSM2021007' 'WSSSM2021008']
                                                    seawater  sediment  biota
Number of rows in dfs :                                21216     39817  15827
Number of rows in tfm.dfs:                             21208     39816  15827
Number of dropped rows:                                    8         1      0
Number of rows in tfm.dfs + Number of dropped r...     21216     39817  15827 



***

### Filtered

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``filtered``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Filtered``*

`get_filtered_lut` reads the file at `filtered_lut_path()` and from the contents of this file creates a dictionary linking `name` to `id`.

In [ ]:
#| export 
def get_filtered_lut():
    df = pd.read_excel(filtered_lut_path(), usecols=['name','id'])
    return df.set_index('name').to_dict()['id']

In [ ]:
get_filtered_lut()

{'Not applicable': -1, 'Not available': 0, 'Yes': 1, 'No': 2}

Create  `renaming_rules` to rename the HELCOM data to the MARIS format.

In [ ]:
renaming_rules = {'N': 'No',
                  'n': 'No',
                  'F': 'Yes'}

`LookupFiltCB` converts the HELCOM `FILT` format to the MARIS `FILT` format.

In [ ]:
# | export
class LookupFiltCB(Callback):
    "Lookup FILT value."
    def __init__(self ,
                 rules=renaming_rules,
                 fn_lut=get_filtered_lut
                 ):
        fc.store_attr()

    def __call__(self, tfm):
        lut = self.fn_lut()
        rules = self.rules
        for grp in tfm.dfs.keys():
            if "FILT" in tfm.dfs[grp].columns:
                # Fill values that are not in the renaming rules with 'Not Available'.
                tfm.dfs[grp].loc[~tfm.dfs[grp]["FILT"].isin(list(rules.keys())), "FILT"] = 'Not available'
                # Rename HELCOM format with MARIS format. 
                tfm.dfs[grp]['FILT'] = tfm.dfs[grp]['FILT'].apply(lambda x : rules[x] if x != 'Not available' else 'Not available')
                # Perform lookup
                tfm.dfs[grp]['FILT'] = tfm.dfs[grp]['FILT'].apply(lambda x : lut[x])                

Apply the transformer for callbacks `LowerStripRdnNameCB`, `RemapRdnNameCB`, `ParseTimeCB`,  `NormalizeUncUnitCB()`, `LookupBiotaSpeciesCB(get_maris_species)`, `LookupBiotaBodyPartCB(get_maris_bodypart)`, `LookupSedimentCB(get_maris_sediments)`, `LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())`, `LookupUnitCB()`,  `LookupDetectionLimitCB` and `LookupFiltCB()`. Then, print the unique `FILT` for the `seawater` dataframe.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            LookupFiltCB()
                            ])

print(tfm()['seawater']['FILT'].unique())

[0 2 1]


***

#### ~~Lookup : Area~~

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``area``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: Area is not included*

TODO : Write callback for area. Will I use the marineregions.org API to complete lookup? 

In [ ]:
'''#| export 
def get_area_lut():
    df = pd.read_excel(area_lut_path(), usecols=['displayName','areaId'])
    return df.set_index('displayName').to_dict()['areaId']
'''

"#| export \ndef get_area_lut():\n    df = pd.read_excel(area_lut_path(), usecols=['displayName','areaId'])\n    return df.set_index('displayName').to_dict()['areaId']\n"

***

### ~~Sample Notes~~

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``sample_notes``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Sample notes
``*

>  HELCOM data does not include ``sample_notes``. 

***

### ~~Measurement Notes~~

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``measurement_notes``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Measurement notes``*

>  HELCOM data does not include ``measurement_notes``. 

***

### Station ID 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: Station ID is not included.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Station ID``*

>  MARIS NetCDF4 format does not include Station ID.

In [ ]:
# | export
class RemapStationIdCB(Callback):
    "Remap Station ID to MARIS format."
    def __init__(self):
        fc.store_attr()

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            tfm.dfs[grp]['station_id'] = tfm.dfs[grp]['STATION']


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB()
                            ])

print(tfm()['seawater']['station_id'].unique())


['RU10' 'RU11' 'RU19' 'RU20' 'RU23' 'RU25' 'RU32' 'RU52' 'RU89' 'RU94*'
 'RU99' 'RU141' 'RU156' 'B6' 'B50' 'BY15' 'BY28' 'L6' 'L7' 'L8' 'LА-11'
 'L13' 'Т1' 'RU5' 'B46' 'P16' 'Z1' 'SM1' 'SM2' 'SM5' 'P116' 'K11' 'P110'
 '2N2' 'ZN2' 'P14' 'P63' 'P140' 'P2' 'P3' 'P5' 'P39' 'B12' 'B13' 'B15'
 'K3' 'M3' 'R4' 'P1' 'P40' 'A' 'GDR250' 'Z' 'K6' 'P127' 'P104' 'NP' '4P7'
 'MW4' 'ZN4' 'EDC19' 'STOLGR' 'SCHLEI' 'KALKGR' 'KLBELT' 'KN' 'EDC20'
 'GBELT2' 'EDC38' 'EDC42' 'EDC32' 'EDC46' 'EDC47' 'KATT1' 'EDC55' 'SOUNDA'
 'SUND2' 'EDC58' 'ARKO1' 'BY2' 'EDC65' 'EDC68' 'GDR213' 'EDC80' 'EDC101'
 'EDC88' '2' 'EDC73' 'BHOLM3' 'EDC63' '18' 'DARSS' 'KOTN11' 'KFOTN6'
 'EDC35' 'EDC16' 'EDC7' 'EDC148' 'FBELT1' 'EDC92' '87/21' 'EDC70' 'EDC66'
 'GDR109' 'GDR069' 'EDC18' 'EDC17' 'EDC39' 'ZZZ' 'EDC93' 'EDC81' 'GBELT1'
 'EDC40' '8' 'EDC145' 'EDC36' '11' 'EDC54' 'EDC53' 'SUND4' 'SUND3' 'SUND1'
 'BHOLM2' 'EDC75' '22' '29' 'EDC821' 'EDC82' 'EDC89' 'EDC98' 'EDC119'
 'EDC114' '33' 'EDC96' 'EDC118' 'EDC109' '37' 'EDC100' '39

***

### Profile ID, Transect ID or Sequence ID

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: Profile ID, Transect ID or Sequence ID is not included.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variable: ``Profile or transect ID``*

>  MARIS NetCDF4 format does not include Profile ID, Transect ID or Sequence ID.

In [ ]:
# | export
class RemapProfileIdCB(Callback):
    "Remap Profile ID to MARIS format."
    def __init__(self):
        fc.store_attr()

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            tfm.dfs[grp]['profile_or_transect_id'] = tfm.dfs[grp]['SEQUENCE']


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB()
                            ])

print(tfm()['seawater']['profile_or_transect_id'].unique())


[2012003. 2012004. 2012005. ...  202106.  202107.  202108.]


***

### Sediment slice position (top and bottom)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: Top and Bottom is not included.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Top`` and ``Bottom``.*

>  MARIS NetCDF4 format does not include sediment slice top and bottom.

In [ ]:
# | export
class RemapSedSliceTopBottomCB(Callback):
    "Remap Sediment slice top and bottom to MARIS format."
    def __init__(self):
        fc.store_attr()

    def __call__(self, tfm):
        tfm.dfs['sediment']['bottom'] = tfm.dfs['sediment']['LOWSLI']
        tfm.dfs['sediment']['top'] = tfm.dfs['sediment']['UPPSLI']

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB()
                            ])

print(tfm()['sediment']['top'].head())


0    15.0
1    20.0
2     0.0
3     2.0
4     4.0
Name: top, dtype: float64


***

### Dry to wet ratio

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: DW% is not included.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Dry/wet ratio``.*

HELCOM Description:

**Sediment:**
1. DW%: DRY WEIGHT AS PERCENTAGE (%) OF FRESH WEIGHT.
2. VALUE_Bq/kg: Measured radioactivity concentration in Bq/kg dry wt. in scientific format(e.g. 123 = 1.23E+02, 0.076 = 7.6E-02)

**Biota:**
1. WEIGHT: Average weight (in g) of specimen in the sample
2. DW%: DRY WEIGHT AS PERCENTAGE (%) OF FRESH WEIGHT

In [ ]:
# | export
class LookupDryWetRatio(Callback):
    "Lookup dry wet ratio and format for MARIS."
    def __init__(self):
        fc.store_attr()

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            if 'DW%' in tfm.dfs[grp].columns:
                tfm.dfs[grp]['dry_wet_ratio'] = tfm.dfs[grp]['DW%']
                # Convert 'Dw%' = 0% to 'nan'.
                tfm.dfs[grp].loc[tfm.dfs[grp]['dry_wet_ratio'] == 0, 'dry_wet_ratio'] = np.NaN

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
                    

print(tfm.dfs['biota']['dry_wet_ratio'].head())


                                                    seawater  sediment  biota
Number of rows in dfs :                                21216     39817  15827
Number of rows in tfm.dfs:                             21208     39816  15827
Number of dropped rows:                                    8         1      0
Number of rows in tfm.dfs + Number of dropped r...     21216     39817  15827 

0    18.453
1    18.453
2    18.453
3    18.453
4    18.458
Name: dry_wet_ratio, dtype: float64


***

### Capture Coordinates

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variables: ``lon``  and ``lat``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Longitude decimal`` and ``Latitude decimal``.*

Use decimal degree coordinates if available; otherwise, convert from degree-minute format to decimal degrees.

In [ ]:
#| export
# Columns of interest coordinates
coi_coordinates = {'seawater' : { 'lon_d' : 'LONGITUDE (dddddd)', 'lat_d':'LATITUDE (dddddd)',
                                 'lon_m' : 'LONGITUDE (ddmmmm)', 'lat_m':'LATITUDE (ddmmmm)'},
                 'biota' : { 'lon_d' : 'LONGITUDE dddddd', 'lat_d':'LATITUDE dddddd',
                                 'lon_m' : 'LONGITUDE ddmmmm', 'lat_m':'LATITUDE ddmmmm'},
                 'sediment': { 'lon_d' : 'LONGITUDE (dddddd)', 'lat_d':'LATITUDE (dddddd)',
                                 'lon_m' : 'LONGITUDE (ddmmmm)', 'lat_m':'LATITUDE (ddmmmm)'}
                 }

In [ ]:
#| export
def ddmmmm2dddddd(ddmmmm):
    mins, degs = modf(ddmmmm)
    # move 2 decimal place
    mins = mins*100
    return round((int(degs)+(float(mins)/60)), 4)

In [ ]:
# | export
class FormatCoordinates(Callback):
    "Format coordinates for MARIS."
    def __init__(self, 
                 coi: dict,
                 fn_convert_cor
                 ):
        fc.store_attr()

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            # If coordinates with format dddddd (e.g. )
            # Get coordinates from dddddd unless dddddd equals 0 or nan. 
            condition = ((tfm.dfs[grp][self.coi[grp]['lon_d']].isna()) | (tfm.dfs[grp][self.coi[grp]['lon_d']] == 0 )) | ((tfm.dfs[grp][self.coi[grp]['lat_d']].isna()) | (tfm.dfs[grp][self.coi[grp]['lat_d']] == 0 ))            
            
            
            tfm.dfs[grp]['lon']  = np.where(condition,
                                            tfm.dfs[grp][self.coi[grp]['lon_m']].apply(lambda x: self.fn_convert_cor(x)),
                                            tfm.dfs[grp][self.coi[grp]['lon_d']])
            
            tfm.dfs[grp]['lat']  = np.where(condition,
                                            tfm.dfs[grp][self.coi[grp]['lat_m']].apply(lambda x: self.fn_convert_cor(x)),
                                            tfm.dfs[grp][self.coi[grp]['lat_d']])      
              

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['biota'][['lat','lon']])

ValueError: cannot convert float NaN to integer

***

### Sanitize coordinates

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variables: ``lon``  and ``lat``*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Longitude decimal`` and ``Latitude decimal``.*

Sanitize coordinates drops a row when both longitude & latitude equal 0 or data contains unrealistic longitude & latitude values. Converts longitude & latitude `,` separator to `.` separator."

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['biota'][['lat','lon']])


***

### Sanitize value

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*NetCDF4 format variable: ``value``.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*Open Refine format variables: ``Activity or MDA``.*

In [ ]:
#| export
# Columns of interest
coi_val = {'seawater' : { 'val' : 'VALUE_Bq/m³'},
                 'biota':  {'val' : 'VALUE_Bq/kg'},
                 'sediment': { 'val' : 'VALUE_Bq/kg'}}

In [ ]:
# | export
class SanitizeValue(Callback):
    "Sanitize value. Remove blank entries."
    def __init__(self,
                coi: dict):
        fc.store_attr()

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            val = self.coi[grp]['val']
            # Keep rows where either value (i.e. VALUE_Bq/m³ or VALUE_Bq/kg ) is not 'nan'
            tfm.dfs[grp] = tfm.dfs[grp][tfm.dfs[grp][[val]].notna().any(axis=1)]


In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SanitizeValue(coi_val),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

In [ ]:
seawater_review=tfm.dfs_dropped['seawater']
biota_review=tfm.dfs_dropped['biota']
sediment_review=tfm.dfs_dropped['sediment']

***

### Review DFS and TFM data

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SanitizeValue(coi_val),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


In [ ]:
seawater_review=tfm.dfs_dropped['seawater']
biota_review=tfm.dfs_dropped['biota']
sediment_review=tfm.dfs_dropped['sediment']

### Columns of interest and rename for NetCDF

> Column names are standardized to MARIS NetCDF format (i.e. PEP8 ). 

In [ ]:
#| export
# Define columns of interest (keys) and renaming rules (values).
def get_renaming_rules_netcdf():
    vars = cdl_cfg()['vars']
    return {('seawater','biota', 'sediment') : {    
                                                        ## DEFAULT
                                                        'lat' : vars['defaults']['lat']['name'] ,
                                                        'lon' : vars['defaults']['lon']['name'] ,
                                                        'time' : vars['defaults']['time']['name'],
                                                        'NUCLIDE' : 'nuclide',
                                                        'unit' : vars['suffixes']['unit']['name'],
                                                        #'station_id' : 'data_provider_station_id',
                                                        #'data_provider_sample_id' : vars['defaults']['data_provider_sample_id']['name'],
                                                        #'profile_or_transect_id' : 'profile_id',
                                                        'detection_limit' : vars['suffixes']['detection_limit']['name']
                                                        #'Sampling method' : 'sampling_method'
                                                        #'Preparation method' : 'preparation_method'
                                                        #'Counting method' : 'counting_method'
                                                        #'Sample notes' : 'sample_notes'
                                                        #'Measurement notes' : 'measurement_notes'
                                                    },
                  ('seawater',) : {
                                ## SEAWATER
                                'VALUE_Bq/m³': 'value',
                                'ERROR%_m³': vars['suffixes']['uncertainty']['name'],
                                'TDEPTH': vars['defaults']['tot_depth']['name'],
                                'SDEPTH': vars['defaults']['smp_depth']['name'],
                                'SALIN' : vars['suffixes']['salinity']['name'],
                                'TTEMP' : vars['suffixes']['temperature']['name'],
                                #'FILT' : vars['suffixes']['filtered']['name']
                                },
                  ('biota',) : { 
                                ## BIOTA
                                'VALUE_Bq/kg': 'value',
                                'ERROR%' : vars['suffixes']['uncertainty']['name'],
                                'species' : vars['bio']['species']['name'],
                                'body_part' : vars['bio']['body_part']['name'],
                                'bio_group' : vars['bio']['bio_group']['name'],
                                'SDEPTH' : vars['defaults']['smp_depth']['name'],
                                #'DW%' : 'dry_wet_ratio'
                                #'Drying Method' : drying_method
                                
                                },
                  ('sediment',) : {
                                ## SEDIMENT
                                'VALUE_Bq/kg': 'value',
                                'ERROR%_kg' : vars['suffixes']['uncertainty']['name'],
                                'TDEPTH' : vars['defaults']['tot_depth']['name'],
                                'sed_type' : vars['sed']['sed_type']['name'],
                                #'top' : 'top',
                                #'bottom' : 'bottom', 
                                #'DW%' : 'dry_wet_ratio'
                                #'Drying Method' : drying_method
                                }
                    }

Open Refine data format includes additional data that is not available in NetCDF format. Here we select columns of interest for Open Refine and standardize columns names to MARIS NetCDF format. 

> For Open Refine CSV column names are standardized to MARIS NetCDF format. 

In [ ]:
#| export
# Define columns of interest (keys) and renaming rules (values) for encoding to open refine csv. .
def get_renaming_rules_openrefine():
    vars = cdl_cfg()['vars']
    return {('seawater','biota', 'sediment') : {    
                                                        ## DEFAULT
                                                        'lat' : vars['defaults']['lat']['name'] ,
                                                        'lon' : vars['defaults']['lon']['name'] ,
                                                        'time' : vars['defaults']['time']['name'],
                                                        'NUCLIDE' : 'nuclide',
                                                        'unit' : vars['suffixes']['unit']['name'],
                                                        'station_id' : 'data_provider_station_id',
                                                        'data_provider_sample_id' : vars['defaults']['data_provider_sample_id']['name'],
                                                        'profile_or_transect_id' : 'profile_id',
                                                        'detection_limit' : vars['suffixes']['detection_limit']['name']
                                                        #'Sampling method' : 'sampling_method'
                                                        #'Preparation method' : 'preparation_method'
                                                        #'Counting method' : 'counting_method'
                                                        #'Sample notes' : 'sample_notes'
                                                        #'Measurement notes' : 'measurement_notes'
                                                    },
                  ('seawater',) : {
                                ## SEAWATER
                                'VALUE_Bq/m³': 'value',
                                'ERROR%_m³': vars['suffixes']['uncertainty']['name'],
                                'TDEPTH': vars['defaults']['tot_depth']['name'],
                                'SDEPTH': vars['defaults']['smp_depth']['name'],
                                'SALIN' : vars['suffixes']['salinity']['name'],
                                'TTEMP' : vars['suffixes']['temperature']['name'],
                                'FILT' : vars['suffixes']['filtered']['name']
                                },
                  ('biota',) : { 
                                ## BIOTA
                                'VALUE_Bq/kg': 'value',
                                'ERROR%' : vars['suffixes']['uncertainty']['name'],
                                'species' : vars['bio']['species']['name'],
                                'body_part' : vars['bio']['body_part']['name'],
                                'bio_group' : vars['bio']['bio_group']['name'],
                                'SDEPTH' : vars['defaults']['smp_depth']['name'],
                                'DW%' : 'dry_wet_ratio'
                                #'Drying Method' : drying_method
                                
                                },
                  ('sediment',) : {
                                ## SEDIMENT
                                'VALUE_Bq/kg': 'value',
                                'ERROR%_kg' : vars['suffixes']['uncertainty']['name'],
                                'TDEPTH' : vars['defaults']['tot_depth']['name'],
                                'sed_type' : vars['sed']['sed_type']['name'],
                                'top' : 'top',
                                'bottom' : 'bottom', 
                                'DW%' : 'dry_wet_ratio'
                                #'Drying Method' : drying_method
                                }
                    }

In [ ]:

#| export
class SelectAndRenameColumnCB(Callback):
    def __init__(self,
                 fn_renaming_rules,
                ):
        fc.store_attr()
    def __call__(self, tfm):
        renaming = self.fn_renaming_rules()
        for grp in tfm.dfs.keys():            
            # get columns related to the grp (e.g. 'biota').
            coi = [v for k, v in renaming.items() if grp in k]
            # Join cols of interest
            coi_rename = {}
            for d in coi:
                for k, v in d.items(): 
                    coi_rename[k]=v
            # list cols
            cols = list(coi_rename.keys()) 
            # select cols in df 
            tfm.dfs[grp] = tfm.dfs[grp].loc[:, cols]
            # Rename cols
            tfm.dfs[grp].rename(columns=coi_rename, inplace=True)
            

In [ ]:

#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SanitizeValue(coi_val),
                            SelectAndRenameColumnCB(get_renaming_rules_netcdf),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()

print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

***

### Reshape: long to wide

Convert data from long to wide and rename columns to comply with NetCDF format.

In [ ]:
#| export
class ReshapeLongToWide(Callback):
    "Convert data from long to wide with renamed columns."
    def __init__(self, columns=['nuclide'], values=['value']):
        fc.store_attr()
        # Retrieve all possible derived vars (e.g 'unc', 'dl', ...) from configs
        self.derived_cols = [value['name'] for value in cdl_cfg()['vars']['suffixes'].values()]
    
    def renamed_cols(self, cols):
        "Flatten columns name"
        return [inner if outer == "value" else f'{inner}{outer}'
                if inner else outer
                for outer, inner in cols]

    def pivot(self, df):
        # Among all possible 'derived cols' select the ones present in df
        derived_coi = [col for col in self.derived_cols if col in df.columns]
        
        df=df.reset_index()
        
        idx = list(set(df.columns) - set(self.columns + derived_coi + self.values))
        
        # Create a fill_value to replace NaN values in the columns used as the index in the pivot table.
        # Check if num_fill_value is already in the dataframe index values. If num_fill_value already exists
        # then increase num_fill_value by 1 until a value is found for num_fill_value that is not in the dataframe. 
        num_fill_value = 99999999999999
        while (df[idx] == num_fill_value).any().any():
            num_fill_value += 1
        # Fill in nan values for each col found in idx. 
        for col in idx:   
            if pd.api.types.is_numeric_dtype(df[col]):
                fill_value = num_fill_value
            if pd.api.types.is_string_dtype(df[col]):
                fill_value = 'NOT AVAILABLE'
                
            df[col]=df[col].fillna(fill_value)

        pivot_df=df.pivot_table(index=idx,
                              columns=self.columns,
                              values=self.values + derived_coi,
                              fill_value=np.nan,
                              aggfunc=lambda x: x
                              ).reset_index()
        
        pivot_df.index.name = 'sample'
        pivot_df=pivot_df.reset_index('sample')
        
        # Replace fill_value  with  np.nan
        pivot_df[idx]=pivot_df[idx].replace({'NOT AVAILABLE': np.nan,
                                             num_fill_value : np.nan})
        return (pivot_df)

    def __call__(self, tfm):
        for grp in tfm.dfs.keys():
            tfm.dfs[grp] = self.pivot(tfm.dfs[grp])
            tfm.dfs[grp].columns = self.renamed_cols(tfm.dfs[grp].columns)

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SanitizeValue(coi_val),
                            SelectAndRenameColumnCB(get_renaming_rules_netcdf),
                            ReshapeLongToWide(), 
                            CompareDfsAndTfm(dfs)

                            ])

print(tfm()['biota'].head())
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

   sample   lat  index   lon  species        time  smp_depth  body_part  \
0       0  53.5  14075  14.3      247  1443052800        NaN         52   
1       1  53.5  14076  14.3      247  1443052800        NaN         52   
2       2  53.5  14077  14.3      247  1443052800        NaN         52   
3       3  53.5  14078  14.3      247  1443052800        NaN         52   
4       4  53.5  14079  14.3      247  1443052800        NaN         52   

   bio_group  ac228_dl  ...  sr89  sr90  tc99  te129m  th228  th232  tl208  \
0          4       NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
1          4       NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
2          4       NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
3          4       NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   
4          4       NaN  ...   NaN   NaN   NaN     NaN    NaN    NaN    NaN   

   u235  zn65  zr95  
0   NaN   NaN   NaN  
1   NaN   NaN   NaN  
2   NaN   NaN 

***

## NetCDF encoder

### Example change logs

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SanitizeValue(coi_val),
                            SelectAndRenameColumnCB(get_renaming_rules_netcdf),
                            ReshapeLongToWide(), 
                            #CompareDfsAndTfm(dfs)
                            ])

# Transform
tfm()
# Check transformation logs
tfm.logs

['Convert nuclide names to lowercase & strip any trailing space(s)',
 'Remap to MARIS radionuclide names.',
 'Encode time as `int` representing seconds since xxx',
 'Convert from relative error % to uncertainty of activity unit',
 '\n    Biota species remapped to MARIS db:\n        CARD EDU: Cerastoderma edule\n        LAMI SAC: Saccharina latissima\n        PSET MAX: Scophthalmus maximus\n        STIZ LUC: Sander luciopercas\n    ',
 "\n    Update bodypart id based on MARIS dbo_bodypar.xlsx:\n        - 3: 'Whole animal eviscerated without head',\n        - 12: 'Viscera',\n        - 8: 'Skin'\n    ",
 '\n    Update biogroup id  based on MARIS dbo_species.xlsx\n    ',
 "\n    Update sediment id  based on MARIS dbo_sedtype.xlsx\n        -99: '(Not available)'\n        - na: '(Not available)'\n        - 56: '(Not available)'\n        - 73: '(Not available)'\n    ",
 'Remap value type to MARIS format.',
 'Remap key to MARIS data_provider_sample_id format.',
 'Remap Station ID to MARIS form

### Feed global attributes

In [ ]:
#| export
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']


In [ ]:
#| export
def get_attrs(tfm, zotero_key, kw=kw):
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(cfg()),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
#|eval: false
get_attrs(tfm, zotero_key=zotero_key, kw=kw)

{'geospatial_lat_min': '31.17',
 'geospatial_lat_max': '65.75',
 'geospatial_lon_min': '9.6333',
 'geospatial_lon_max': '53.5',
 'geospatial_bounds': 'POLYGON ((9.6333 53.5, 31.17 53.5, 31.17 65.75, 9.6333 65.75, 9.6333 53.5))',
 'time_coverage_start': '1984-01-10T00:00:00',
 'time_coverage_end': '2018-12-14T00:00:00',
 'title': 'Environmental database - Helsinki Commission Monitoring of Radioactive Substances',
 'summary': 'MORS Environment database has been used to collate data resulting from monitoring of environmental radioactivity in the Baltic Sea based on HELCOM Recommendation 26/3.\n\nThe database is structured according to HELCOM Guidelines on Monitoring of Radioactive Substances (https://www.helcom.fi/wp-content/uploads/2019/08/Guidelines-for-Monitoring-of-Radioactive-Substances.pdf), which specifies reporting format, database structure, data types and obligatory parameters used for reporting data under Recommendation 26/3.\n\nThe database is updated and quality assured annua

In [ ]:
#| export
def enums_xtra(tfm, vars):
    "Retrieve a subset of the lengthy enum as 'species_t' for instance"
    enums = Enums(lut_src_dir=lut_path(), cdl_enums=cdl_cfg()['enums'])
    xtras = {}
    for var in vars:
        unique_vals = tfm.unique(var)
        if unique_vals.any():
            xtras[f'{var}_t'] = enums.filter(f'{var}_t', unique_vals)
    return xtras

### Encoding NETCDF

In [ ]:
#| export
def encode(fname_in, fname_out_nc, nc_tpl_path, **kwargs):
    dfs = load_data(fname_in)
    tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                                RemapRdnNameCB(),
                                ParseTimeCB(),
                                EncodeTimeCB(cfg()),                             
                                NormalizeUncUnitCB(),
                                LookupBiotaSpeciesCB(get_maris_species),
                                LookupBiotaBodyPartCB(get_maris_bodypart), 
                                LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                                LookupSedimentCB(get_maris_sediments),
                                LookupUnitCB(),
                                LookupDetectionLimitCB(),
                                RemapDataProviderSampleIdCB(),
                                RemapStationIdCB(),
                                RemapProfileIdCB(), 
                                RemapSedSliceTopBottomCB(),
                                LookupDryWetRatio(),
                                FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                                SanitizeLonLatCB(),
                                SanitizeValue(coi_val),
                                SelectAndRenameColumnCB(get_renaming_rules_netcdf),
                                ReshapeLongToWide()
                                ])
    tfm()
    encoder = NetCDFEncoder(tfm.dfs, 
                            src_fname=nc_tpl_path,
                            dest_fname=fname_out_nc, 
                            global_attrs=get_attrs(tfm, zotero_key=zotero_key, kw=kw),
                            verbose=kwargs.get('verbose', False),
                            enums_xtra=enums_xtra(tfm, vars=['species', 'body_part'])
                           )
    encoder.encode()

In [ ]:
#|eval: false
encode(fname_in, fname_out_nc, nc_tpl_path(), verbose=False)

## Data transformation pipeline for Open Refine CSV. 

Data intended for the ``Open Refine`` format is first processed using several callbacks from the ``Data transformation pipeline for NetCDF``.

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SanitizeValue(coi_val),
                            SelectAndRenameColumnCB(get_renaming_rules_openrefine), 
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(), 
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB(), 
                            LookupUnitByIdCB(), 
                            LookupValueTypeByIdCB(), 
                            LookupSpeciesByIdCB(),
                            LookupBodypartByIdCB(), 
                            LookupSedimentTypeByIdCB(),
                            CompareDfsAndTfm(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(f"Seawater cols: {tfm.dfs['seawater'].columns}")
print(f"Sediment cols: {tfm.dfs['sediment'].columns}")
print(f"Biota cols: {tfm.dfs['biota'].columns}")

                                                    seawater  sediment  biota
Number of rows in dfs :                                20318     37347  14893
Number of rows in tfm.dfs:                             20242     37089  14873
Number of dropped rows:                                   76       258     20
Number of rows in tfm.dfs + Number of dropped r...     20318     37347  14893 

Seawater cols: Index(['lat', 'lon', 'time', 'nuclide', '_unit', 'data_provider_station_id',
       'data_provider_sample_id', 'profile_id', '_dl', 'value', '_unc',
       'tot_depth', 'smp_depth', '_sal', '_temp', '_filt',
       'Sampling start date', 'Sampling start time', 'Sample type', 'Nuclide',
       'Latitude degrees', 'Latitude minutes', 'Latitude seconds',
       'Latitude direction', 'Longitude degrees', 'Longitude minutes',
       'Longitude seconds', 'Longitude direction', 'Unit', 'Value type'],
      dtype='object')
Sediment cols: Index(['lat', 'lon', 'time', 'nuclide', '_unit', 'data_pr

In [ ]:
tfm.dfs['sediment'].columns

Index(['lat', 'lon', 'time', 'nuclide', '_unit', 'data_provider_station_id',
       'data_provider_sample_id', 'profile_id', '_dl', 'value', '_unc',
       'tot_depth', 'sed_type', 'top', 'bottom', 'dry_wet_ratio',
       'Sampling start date', 'Sampling start time', 'Sample type', 'Nuclide',
       'Latitude degrees', 'Latitude minutes', 'Latitude seconds',
       'Latitude direction', 'Longitude degrees', 'Longitude minutes',
       'Longitude seconds', 'Longitude direction', 'Unit', 'Value type',
       'Sediment type'],
      dtype='object')

In [ ]:
#| export` is a special comment tag that is often used in code documentation or code generation tools to indicate that the following code block should be exported or made available for external use. This can be useful for generating documentation, extracting specific parts of code for reuse, or for other automated processes that require specific code sections to be identified.
#| export
# Define columns of interest (keys) and renaming rules (values).
def get_renaming_rules_netcdf2OpenRefine():
    vars = cdl_cfg()['vars']
    return {('seawater','biota', 'sediment') : {    
                                                        ## DEFAULT
                                                        'Sample type' : 'Sample type',
                                                        'Latitude degrees' : 'Latitude degrees',
                                                        'Latitude minutes' : 'Latitude minutes',
                                                        'Latitude seconds' : 'Latitude seconds',
                                                        'Latitude direction' : 'Latitude direction',
                                                        'Longitude degrees' : 'Longitude degrees',
                                                        'Longitude minutes' : 'Longitude minutes',
                                                        'Longitude seconds' : 'Longitude seconds', 
                                                        'Longitude direction' : 'Longitude direction', 
                                                        vars['defaults']['lat']['name'] : 'Latitude decimal',
                                                        vars['defaults']['lon']['name'] : 'Longitude decimal',
                                                        'Sampling start date' : 'Sampling start date',
                                                        'Sampling start time' : 'Sampling start time',
                                                        'Nuclide' : 'Nuclide',
                                                        'Value type': 'Value type',
                                                        'Unit' : 'Unit',
                                                        'value' : 'Activity or MDA',
                                                        vars['suffixes']['uncertainty']['name'] : 'Uncertainty',
                                                        'data_provider_station_id' : 'Station ID',
                                                        vars['defaults']['data_provider_sample_id']['name'] :'Sample ID',
                                                        'profile_id' : 'Profile or transect ID',                                                        
                                                        #'Sampling method' : 'sampling_method'
                                                        #'Preparation method' : 'preparation_method'
                                                        #'Counting method' : 'counting_method'
                                                        #'Sample notes' : 'sample_notes'
                                                        #'Measurement notes' : 'measurement_notes'
                                                    },
                  ('seawater',) : {
                                ## SEAWATER
                                vars['defaults']['tot_depth']['name'] : 'Total depth',
                                vars['defaults']['smp_depth']['name'] : 'Sampling depth' ,
                                vars['suffixes']['salinity']['name'] : 'Salinity',
                                vars['suffixes']['temperature']['name'] : 'Temperature',
                                vars['suffixes']['filtered']['name'] : 'Filtered'
                                },
                  ('biota',) : { 
                                ## BIOTA
                                'Species' : 'Species',
                                'Body part' : 'Body part',
                                #'bio_group' : vars['bio']['bio_group']['name'],
                                #'SDEPTH' : vars['defaults']['smp_depth']['name'],
                                'dry_wet_ratio' : 'Dry/wet ratio'
                                #'Drying Method' : drying_method
                                
                                },
                  ('sediment',) : {
                                ## SEDIMENT
                                vars['defaults']['tot_depth']['name'] : 'Total depth',
                                'Sediment type' : 'Sediment type',
                                'top' : 'Top',
                                'bottom' : 'Bottom', 
                                'dry_wet_ratio' : 'Dry/wet ratio'
                                #'Drying Method' : drying_method
                                }
                    }

In [ ]:
#|eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeValue(coi_val),
                            SanitizeLonLatCB(),
                            SelectAndRenameColumnCB(get_renaming_rules_openrefine), 
                            LookupTimeFromEncodedTime(cfg()),
                            GetSampleTypeCB(), 
                            LookupNuclideByIdCB(),
                            ConvertLonLatCB(), 
                            LookupUnitByIdCB(), 
                            LookupValueTypeByIdCB(), 
                            LookupSpeciesByIdCB(),
                            LookupBodypartByIdCB(), 
                            LookupSedimentTypeByIdCB(),
                            CompareDfsAndTfm(dfs), 
                            SelectAndRenameColumnCB(get_renaming_rules_netcdf2OpenRefine)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(f"Seawater cols: {tfm.dfs['seawater'].columns}")
print(f"Sediment cols: {tfm.dfs['sediment'].columns}")
print(f"Biota cols: {tfm.dfs['biota'].columns}")

                                                    seawater  sediment  biota
Number of rows in dfs :                                20318     37347  14893
Number of rows in tfm.dfs:                             20242     37089  14873
Number of dropped rows:                                   76       258     20
Number of rows in tfm.dfs + Number of dropped r...     20318     37347  14893 

Seawater cols: Index(['Sample type', 'Latitude degrees', 'Latitude minutes',
       'Latitude seconds', 'Latitude direction', 'Longitude degrees',
       'Longitude minutes', 'Longitude seconds', 'Longitude direction',
       'Latitude decimal', 'Longitude decimal', 'Sampling start date',
       'Sampling start time', 'Nuclide', 'Value type', 'Unit',
       'Activity or MDA', 'Uncertainty', 'Station ID', 'Sample ID',
       'Profile or transect ID', 'Total depth', 'Sampling depth', 'Salinity',
       'Temperature', 'Filtered'],
      dtype='object')
Sediment cols: Index(['Sample type', 'Latitude degre

### Encoding Open Refine CSV

In [ ]:
seawater_review=tfm.dfs['seawater']
biota_review=tfm.dfs['biota']
sediment_review=tfm.dfs['sediment']

In [ ]:
#| export
def encode_open_refine_csv(fname_in, fname_out, ref_id=-1, **kwargs):
    dfs = load_data(fname_in)
    tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                                RemapRdnNameCB(),
                                ParseTimeCB(),
                                EncodeTimeCB(cfg()),                             
                                NormalizeUncUnitCB(),
                                LookupBiotaSpeciesCB(get_maris_species),
                                LookupBiotaBodyPartCB(get_maris_bodypart), 
                                LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                                LookupSedimentCB(get_maris_sediments),
                                LookupUnitCB(),
                                LookupDetectionLimitCB(),
                                RemapDataProviderSampleIdCB(),
                                RemapStationIdCB(),
                                RemapProfileIdCB(), 
                                RemapSedSliceTopBottomCB(),
                                LookupDryWetRatio(),
                                FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                                SanitizeLonLatCB(),
                                SanitizeValue(coi_val),
                                SelectAndRenameColumnCB(get_renaming_rules_openrefine), 
                                LookupTimeFromEncodedTime(cfg()),
                                GetSampleTypeCB(), 
                                LookupNuclideByIdCB(),
                                ConvertLonLatCB(), 
                                LookupUnitByIdCB(), 
                                LookupValueTypeByIdCB(), 
                                LookupSpeciesByIdCB(),
                                LookupBodypartByIdCB(), 
                                LookupSedimentTypeByIdCB(),
                                SelectAndRenameColumnCB(get_renaming_rules_netcdf2OpenRefine)
                                ])
    
    encoder = OpenRefineCsvEncoder(tfm(), 
                            dest_fname=fname_out,
                            ref_id = ref_id,
                            **kwargs)
    encoder.encode()

In [ ]:
encode_open_refine_csv(fname_in, fname_out_csv, ref_id, verbose=False)

## TODO

TODO: Include FILT for NetCDF

TODO : Do we want to include laboratory code in NetCDF?

TODO: Check sediment 'DW%' data that is less than 1%. Is this realistic? Check the 'DW%' data that is 0%. Run below before SelectAndRenameColumnCB. 

In [ ]:
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripRdnNameCB(),
                            RemapRdnNameCB(),
                            ParseTimeCB(),
                            EncodeTimeCB(cfg()),                             
                            NormalizeUncUnitCB(),
                            LookupBiotaSpeciesCB(get_maris_species),
                            LookupBiotaBodyPartCB(get_maris_bodypart), 
                            LookupBiogroupCB(partial(get_biogroup_lut, species_lut_path())),
                            LookupSedimentCB(get_maris_sediments),
                            LookupUnitCB(),
                            LookupDetectionLimitCB(),
                            RemapDataProviderSampleIdCB(),
                            RemapStationIdCB(),
                            RemapProfileIdCB(), 
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetRatio(),
                            FormatCoordinates(coi_coordinates, ddmmmm2dddddd),
                            SanitizeLonLatCB(),
                            SanitizeValue(coi_val)
                            ])
tfm()

{'seawater':                 KEY NUCLIDE  METHOD < VALUE_Bq/m³  VALUE_Bq/m³  ERROR%_m³  \
 0      WKRIL2012003   cs137     NaN           NaN          5.3     1.6960   
 1      WKRIL2012004   cs137     NaN           NaN         19.9     3.9800   
 2      WKRIL2012005   cs137     NaN           NaN         25.5     5.1000   
 3      WKRIL2012006   cs137     NaN           NaN         17.0     4.9300   
 4      WKRIL2012007   cs137     NaN           NaN         22.2     3.9960   
 ...             ...     ...     ...           ...          ...        ...   
 20313  WDHIG2015227    sr90  DHIG02           NaN          6.6     0.4950   
 20314  WDHIG2015237    sr90  DHIG02           NaN          6.9     0.5175   
 20315  WDHIG2015239    sr90  DHIG02           NaN          6.8     0.5100   
 20316  WDHIG2015255    sr90  DHIG02           NaN          7.3     0.5475   
 20317  WDHIG2015256    sr90  DHIG02           NaN          5.5     0.4180   
 
          DATE_OF_ENTRY_x  COUNTRY LABORATORY  SEQ

In [ ]:
grp='sediment'
check_data_sediment=tfm.dfs[grp][(tfm.dfs[grp]['DW%'] < 1) & (tfm.dfs[grp]['DW%'] > 0.001) ]
#check_data_sediment

In [ ]:
grp='sediment'
check_data_sediment=tfm.dfs[grp][(tfm.dfs[grp]['DW%'] == 0) ]
check_data_sediment

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,ERROR%_kg,< VALUE_Bq/m²,VALUE_Bq/m²,ERROR%_m²,DATE_OF_ENTRY_x,...,unit,detection_limit,data_provider_sample_id,station_id,profile_or_transect_id,bottom,top,dry_wet_ratio,lon,lat
9824,SERPC1997001,cs134,NaN,NaN,3.80,0.7600,NaN,5.75,NaN,NaN,...,4,1,SERPC1997001,EE17,1997001.0,2.0,0.0,NaN,25.0167,59.7167
9825,SERPC1997001,cs137,NaN,NaN,389.00,15.5600,NaN,589.00,NaN,NaN,...,4,1,SERPC1997001,EE17,1997001.0,2.0,0.0,NaN,25.0167,59.7167
9826,SERPC1997002,cs134,NaN,NaN,4.78,0.6214,NaN,12.00,NaN,NaN,...,4,1,SERPC1997002,EE17,1997002.0,4.0,2.0,NaN,25.0167,59.7167
9827,SERPC1997002,cs137,NaN,NaN,420.00,16.8000,NaN,1060.00,NaN,NaN,...,4,1,SERPC1997002,EE17,1997002.0,4.0,2.0,NaN,25.0167,59.7167
9828,SERPC1997003,cs134,NaN,NaN,3.12,0.5304,NaN,12.00,NaN,NaN,...,4,1,SERPC1997003,EE17,1997003.0,6.0,4.0,NaN,25.0167,59.7167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15257,SKRIL1999062,th228,1,NaN,68.00,NaN,NaN,NaN,NaN,NaN,...,4,0,SKRIL1999062,RU23,1999062.0,15.0,10.0,NaN,25.5000,59.8167
15258,SKRIL1999063,k40,1,NaN,1210.00,NaN,NaN,NaN,NaN,NaN,...,4,0,SKRIL1999063,RU23,1999063.0,21.5,15.0,NaN,25.5000,59.8167
15259,SKRIL1999063,ra226,KRIL01,NaN,56.50,NaN,NaN,NaN,NaN,NaN,...,4,0,SKRIL1999063,RU23,1999063.0,21.5,15.0,NaN,25.5000,59.8167
15260,SKRIL1999063,ra228,KRIL01,NaN,72.20,NaN,NaN,NaN,NaN,NaN,...,4,0,SKRIL1999063,RU23,1999063.0,21.5,15.0,NaN,25.5000,59.8167


In [ ]:
grp='biota'
check_data_sediment=tfm.dfs[grp][(tfm.dfs[grp]['DW%'] == 0) ]
check_data_sediment

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,BASIS,ERROR%,NUMBER,DATE_OF_ENTRY_x,COUNTRY,...,body_part,bio_group,unit,detection_limit,data_provider_sample_id,station_id,profile_or_transect_id,dry_wet_ratio,lon,lat
5971,BERPC1997002,k40,NaN,NaN,116.00,W,3.4800,NaN,NaN,91.0,...,52,4,5,1,BERPC1997002,Paldiski,1997002,NaN,24.1667,59.3667
5972,BERPC1997002,cs137,NaN,NaN,12.60,W,0.5040,NaN,NaN,91.0,...,52,4,5,1,BERPC1997002,Paldiski,1997002,NaN,24.1667,59.3667
5973,BERPC1997002,cs134,NaN,NaN,0.14,W,0.0252,NaN,NaN,91.0,...,52,4,5,1,BERPC1997002,Paldiski,1997002,NaN,24.1667,59.3667
5974,BERPC1997001,k40,NaN,NaN,116.00,W,4.6400,NaN,NaN,91.0,...,52,4,5,1,BERPC1997001,Paldiski,1997001,NaN,24.1667,59.3667
5975,BERPC1997001,cs137,NaN,NaN,12.00,W,0.4800,NaN,NaN,91.0,...,52,4,5,1,BERPC1997001,Paldiski,1997001,NaN,24.1667,59.3667
5976,BERPC1997001,cs134,NaN,NaN,0.21,W,0.0504,NaN,NaN,91.0,...,52,4,5,1,BERPC1997001,Paldiski,1997001,NaN,24.1667,59.3667


TODO: Note weight definition in HELCOM is different than in MARIS.  HELCOM definition is 'Average weight (in g) of specimen in the sample. MARIS definition is 'dry weight of biota sample in grammes (g).' or 'wet weight of biota sample in grammes (g).'.

TODO :  For biota we have some entries that include a YEAR but no month and day (see tfm.dfs_dropped['biota'][['YEAR', 'MONTH', 'DAY', 'DATE']]). For these entries I have included a day and month as 1`. 

TODO: Review the format of 'ddmmmm'.. Example of data '29.2000'. Assuming that this is 29 degrees. Then 20 minutes with a remainder of .00 minutes. 

TODO: Follow up with HELCOM on entries where the value is nan. 

TODO: Review Maris Open Refine date format. The description format is 'DD-MMM-YYYY'. The example format is '29/Sep/2008'.  